Training Notebook for the G2Net competition. This implements a Bi-directional GRU using Keras, using preprocessed spectrogram.

This uses Yasufumi Nakama's spectrogram preprocessing notebooks and datasets:
* Train: [Notebook](https://www.kaggle.com/yasufuminakama/g2net-spectrogram-generation-train), [Dataset](https://www.kaggle.com/yasufuminakama/g2net-n-mels-128-train-images)
* Test: [Notebook](https://www.kaggle.com/yasufuminakama/g2net-spectrogram-generation-test), [Dataset](https://www.kaggle.com/yasufuminakama/g2net-n-mels-128-test-images)

In [ ]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
class CustomDataset(tf.keras.utils.Sequence):
    def __init__(self, df, directory, batch_size=32, random_state=42, shuffle=True, target=True, ext='.npy'):
        np.random.seed(random_state)
        
        self.directory = directory
        self.df = df
        self.shuffle = shuffle
        self.target = target
        self.batch_size = batch_size
        self.ext = ext
        
        self.on_epoch_end()
    
    def __len__(self):
        return np.ceil(self.df.shape[0] / self.batch_size).astype(int)
    
    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        batch = self.df[start_idx: start_idx + self.batch_size]
        
        signals = []

        for fname in batch.id:
            path = os.path.join(self.directory, fname + self.ext)
            data = np.load(path)
            signals.append(data)
        
        signals = np.stack(signals).astype('float32')
        
        if self.target:
            return signals, batch.target.values
        else:
            return signals
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [ ]:
def build_model():
    inputs = layers.Input(shape=(27, 128))

    gru1 = layers.Bidirectional(layers.GRU(128, return_sequences=True), name='gru_1')
    gru2 = layers.Bidirectional(layers.GRU(128, return_sequences=True), name='gru_2')
    pool1 = layers.GlobalAveragePooling1D(name='avg_pool')
    pool2 = layers.GlobalMaxPooling1D(name='max_pool')

    x = gru1(inputs)
    x = gru2(x)
    x = tf.keras.layers.Concatenate()([pool1(x), pool2(x)])
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dense(1, activation="sigmoid", name="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)
    
    return model

In [ ]:
train = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
sub = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')
train.head()

In [ ]:
sample_df = train.sample(frac=1).reset_index(drop=True)

split = int(sample_df.shape[0] * 0.8)
train_df = sample_df[:split]
valid_df = sample_df[split:]

In [ ]:
train_dset = CustomDataset(
    train_df, '../input/g2net-n-mels-128-train-images', batch_size=64)

valid_dset = CustomDataset(
    valid_df, '../input/g2net-n-mels-128-train-images', batch_size=64, shuffle=False)

test_dset = CustomDataset(
    sub, "../input/g2net-n-mels-128-test-images", batch_size=64, target=False, shuffle=False)

In [ ]:
model = build_model()
model.compile("adam", loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC()])
model.summary()

In [ ]:
ckpt = tf.keras.callbacks.ModelCheckpoint(
    "model_weights.h5", save_best_only=True, save_weights_only=True,
)

train_history = model.fit(
    train_dset, 
    use_multiprocessing=True, 
    workers=4, 
    epochs=10,
    validation_data=valid_dset,
    callbacks=[ckpt],
)

In [ ]:
model.load_weights('model_weights.h5')

In [ ]:
y_pred = model.predict(
    test_dset, use_multiprocessing=True, workers=4, verbose=1
)

In [ ]:
sub['target'] = y_pred
sub.to_csv('submission.csv', index=False)